<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/good_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install exchange_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler

In [50]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [57]:
class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99], pre_period=0):
    self.period = period
    self.increased = increased
    self.traded = traded
    self.pre_period = pre_period # 시작기점

    self.price_dic = {}

    self.start, self.today = self.get_date()
    self.df = self.get_stock_df()
    self.filtered_df = self.get_filtered_df()
    self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    if self.pre_period != 0:
      today = (datetime.date.today() - datetime.timedelta(days=self.pre_period)).strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_position(self, df):
    return df['Close'][-1]/df['Close'].max()

  def get_future_mdd(self, s):
    peak = s.iloc[0]
    max_drawdown = 0
    for price in s:
        if price > peak:
            peak = price
        drawdown = (peak - price) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown

  def get_result_df(self):
    for i in self.filtered_df['종목코드']:
      self.price_dic[i] = fdr.DataReader(str(i), self.start, self.today)

    if (self.pre_period != 0):
      future_dic = {}
      for i in self.filtered_df['종목코드']:
        future = (datetime.date.today() + datetime.timedelta(days=self.pre_period)).strftime('%Y%m%d')
        future_dic[i] = fdr.DataReader(str(i), self.today, future)
      self.filtered_df['mdd'] = [round(self.get_future_mdd(future_dic[x]['Close']), 4) for x in future_dic]
      self.filtered_df['Performance'] = [round((future_dic[x]['Close'][-1]/future_dic[x]['Close'][0]), 4) for x in future_dic]
      self.filtered_df['preperiod'] = [self.pre_period for x in future_dic]
    else:
      self.filtered_df['mdd'] = [0 for x in self.price_dic]
      self.filtered_df['Performance'] = [0 for x in self.price_dic]
      self.filtered_df['preperiod'] = [self.pre_period for x in self.price_dic]

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Position'] = [self.get_position(self.price_dic[x]) for x in self.price_dic]

    t = self.filtered_df.sort_values('Sharpe', ascending=False)

    scaler = MinMaxScaler(feature_range=(5, 10))

    t['Position'] = scaler.fit_transform(t[['Position']]).round(1)
    t['Sortino'] = scaler.fit_transform(t[['Sortino']]).round(1)
    t['Sharpe'] = scaler.fit_transform(t[['Sharpe']]).round(1)
    t['거래대금'] = scaler.fit_transform(t[['거래대금']]).round(1)
    t['등락률'] = scaler.fit_transform(t[['등락률']]).round(1)
    t['합산'] = t['Position'] + t['Sortino'] + t['Sharpe'] + t['거래대금'] + t['등락률']
    t['합산'] = scaler.fit_transform(t[['합산']]).round(1)

    t = t[['종목명', '종료일 종가', '등락률', '거래대금',	'Sharpe',	'Sortino',	'Position', '합산', 'Performance']].reset_index(drop=True).rename({'종료일 종가':'현재가', '등락률':'파워', '거래대금':'관심도', 'Sharpe':'Risk1', 'Sortino':'Risk2', 'Position':'모멘텀'}, axis=1)
    t.index+=1
    t.index.name='순위'

    return t

In [52]:
# k
stocklist = StockList(250, [.99,1], [.99,1])
t = stocklist.filtered_df
t = t[['종목명','등락률','거래대금']].reset_index(drop=True)
t.index+=1
t.index.name='순위'
t

,종목명,등락률,거래대금
순위,,,
1,금양,460.33,29259747183100
2,에코프로,409.20,63292001099600
3,레인보우로보틱스,281.73,18534145689900


In [53]:
from google.colab import files

t.to_csv('king_df.csv', encoding="utf-8-sig") 
files.download('king_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
# Ai
stocklist = StockList()
t = stocklist.result_df
t

,종목명,현재가,파워,관심도,Risk1,Risk2,모멘텀,합산,Performance
순위,,,,,,,,,
1,에스앤에스텍,48850,7.0,5.2,10.0,9.4,9.5,8.9,0
2,메디톡스,248000,6.0,5.6,9.7,9.6,9.2,8.6,0
3,포스코DX,12020,5.0,8.4,8.7,10.0,7.0,8.3,0
4,코스모신소재,171100,9.0,10.0,8.6,8.2,8.8,10.0,0
5,성우하이텍,10190,5.5,6.8,8.1,8.7,10.0,8.3,0
6,에스피지,31650,5.4,8.0,7.9,8.0,8.1,7.8,0
7,에코플라스틱,7230,8.2,6.7,7.7,8.4,9.6,8.7,0
8,루닛,68700,5.1,5.3,7.7,7.6,10.0,7.2,0
9,삼아알미늄,74400,8.5,5.8,7.6,8.4,7.7,7.9,0


In [55]:
t.to_csv('ai_df.csv', encoding="utf-8-sig") 
files.download('ai_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
# performance 
all_df = []
for p in [20, 60, 120]:
  stocklist = StockList(period=250, increased=[.9, .99], traded=[.9, .99], pre_period=p)
  t = stocklist.filtered_df
  t = t[t['Performance'] > 1.01]
  t = t[['종목명', '종료일 종가', 'mdd', 'Performance', 'preperiod']].reset_index(drop=True)
  t.index+=1
  t.index.name='순위'
  all_df.append(t)
t = pd.concat(all_df)
t = t.groupby("preperiod").apply(lambda x: x.sort_values("Performance", ascending=False).head(4)).sample(frac=1).reset_index(drop=True)
t.index.name='순위'
t

,종목명,종료일 종가,mdd,Performance,preperiod
순위,,,,,
0,솔트룩스,24400,0.0743,1.1004,20
1,STX,7110,0.1673,1.4318,20
2,지엔원에너지,4160,0.3905,3.1202,120
3,강원에너지,8560,0.5000,2.3364,120
4,포스코엠텍,11130,0.3720,1.8643,60
5,지엔원에너지,6460,0.3905,2.0093,60
6,이엔플러스,5000,0.3540,2.0400,120
7,삼아알미늄,41150,0.2909,1.8080,120
8,KBG,15340,0.1832,1.1213,20


In [59]:
t.to_csv('performance_df.csv', encoding="utf-8-sig") 
files.download('performance_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data = {'positive': [34, 42, 34, 33, 11, 77, 23],
        'negative': [12, 35, 12, 55, 11, 13, 52],
        'compound': [22, 10, 6, 10, 23, 84, 35]}

df = pd.DataFrame(data, index=['2022/11/11', '2022/11/12', '2022/11/13', '2022/11/14', '2022/11/15', '2022/11/16', '2022/11/17'])
df.index.name = 'date'
df

In [ ]:
df.to_csv('sentiment_df.csv', encoding="utf-8-sig") 
files.download('sentiment_df.csv')